In [1]:
pip install pyspark==3.0.0

Note: you may need to restart the kernel to use updated packages.


In [22]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [23]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import os


spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("Test it 2") \
    .getOrCreate()

sc = spark.sparkContext

In [24]:
sc.stop()

In [25]:
from pymongo import MongoClient
from pprint import pprint as pp
client = MongoClient('mongodb://mongodb:27017')
db = client.ecommerce

In [26]:
res = db.inventory.insert_one(
    {
        "Item": "Ruler",
        "Colors": ["Red","Green","Blue","Clear","Yellow"],
        "Inventory": {
            "OnHand": 47,
            "MinOnHand": 40
        },
        "UnitPrice": 0.89
    }
)

In [45]:
from pyspark.sql import SparkSession

working_directory = 'jars/*'

spark = SparkSession.\
        builder.\
        appName("pyspark-with-mongo").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "1g").\
        config("spark.mongodb.input.uri","mongodb://mongodb:27017/datascience.movies").\
        config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.0").\
        config('spark.driver.extraClassPath', working_directory). \
        getOrCreate()

sc = spark.sparkContext

In [44]:
sc.stop()

In [29]:
next(db.inventory.find())

{'_id': ObjectId('6001ab2b0b963b158fc33cb3'),
 'Item': 'Ruler',
 'Colors': ['Red', 'Green', 'Blue', 'Clear', 'Yellow'],
 'Inventory': {'OnHand': 47, 'MinOnHand': 40},
 'UnitPrice': 0.89}

In [30]:
sc

<SparkContext master=spark://spark-master:7077 appName=pyspark-with-mongo>

In [31]:
df = spark.read.format("com.mongodb.spark.sql.DefaultSource")\
    .option("uri", "mongodb://mongodb:27017/ecommerce.inventory")\
    .load()

Py4JJavaError: An error occurred while calling o284.load.
: java.lang.ClassNotFoundException: Failed to find data source: com.mongodb.spark.sql.DefaultSource. Please find packages at http://spark.apache.org/third-party-projects.html
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:674)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:728)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:230)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:203)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357

In [ ]:
df.select('*').show()

In [ ]:
df.registerTempTable("inventory")

In [ ]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [ ]:
items = sqlContext.sql('''
    select `_id`, `inventory`, `UnitPrice`, `Item` from inventory
''')

In [ ]:
items.take(1)

In [ ]:
items.printSchema()

https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions

In [ ]:
from pyspark.sql.functions import *

items.select('_id', 'inventory', column('UnitPrice').alias('price')).show()

In [ ]:
items.select(current_date()).show()

In [ ]:
items.select("_id", "Item", "inventory.OnHand").show()

In [ ]:
sc.stop()